# Job recommendation

The skills of an individual can be compared with the skills of job profiles giving an indication of how easy it would be to transition to a particular job.

This notebook will demonstrate this functionality using the users and profiles created in the setup notebook. If you have not already gone through the setup notebook please do so before running the code below.

In [23]:
import requests

# If using the docker-compose file lsgraph will be available on localhost port 5000
base_url = "http://localhost:5000"

In [7]:
# Get organisation and graph details

r = requests.get(base_url + "/api/v1/graphs")
print("Status code", r.status_code)

print("Graph response", r.json())

graph_tag = "test"
assert graph_tag in [i['graph_tag'] for i in r.json()], "Graph not found, run setup notebook"

r = requests.get(base_url + "/api/v1/organisations")
print("Status code", r.status_code)

print("Organisation response", r.json())

# Get organisation ID for the test organisation created during setup
assert 'Test organisation' in [i['name'] for i in r.json()], "Organisation not found, run setup notebook"
organisation_id = [i for i in r.json() if i["name"] == "Test organisation"][0]["id"]

Status code 200
Response [{'graph_tag': 'test', 'id': 1, 'name': 'Test graph'}]
Status code 200
Response [{'id': 1, 'name': 'Test organisation'}]


In [16]:
# Get users

r = requests.get(base_url + "/api/v1/organisations/{0}/users".format(organisation_id))
print("Status code", r.status_code)

print("Response", r.json())
all_users = r.json()

Status code 200
Response [{'id': 1, 'name': 'Adam'}, {'id': 2, 'name': 'Beth'}, {'id': 3, 'name': 'Chris'}, {'id': 4, 'name': 'Dani'}]


In [17]:
# Get user profiles

user_profiles = {}
for user in all_users:
    r = requests.get(base_url + "/api/v1/organisations/{0}/users/{1}".format(organisation_id, user["id"]))
    user_detail = r.json()
    user_profiles[user_detail["name"]] = user_detail["profile_id"]

In [18]:
user_profiles

{'Adam': 30, 'Beth': 31, 'Chris': 32, 'Dani': 33}

In [20]:
# Get profiles

r = requests.get(base_url + "/api/v1/organisations/{0}/profiles".format(organisation_id))
print("Status code", r.status_code)

print("Response", r.json())
all_profiles = {i["id"]:i["name"] for i in r.json()}

Status code 200
Response [{'id': 1, 'name': 'Applications Engineer'}, {'id': 2, 'name': 'Data Scientist'}, {'id': 3, 'name': 'Computer and Information Systems Managers'}, {'id': 4, 'name': 'Web Developers'}, {'id': 5, 'name': 'Software Engineer'}, {'id': 6, 'name': 'Software Engineer ML'}, {'id': 7, 'name': 'Systems Engineer'}, {'id': 8, 'name': 'DevOps Engineer'}, {'id': 9, 'name': 'Machine Learning Researcher'}, {'id': 10, 'name': 'Business Intelligence Analyst'}, {'id': 11, 'name': 'Software Developers, Applications'}, {'id': 12, 'name': 'Cloud architect'}, {'id': 13, 'name': 'Sales Engineer'}, {'id': 14, 'name': 'Database Administrators'}, {'id': 15, 'name': 'Security Engineer'}, {'id': 16, 'name': 'Computer Hardware Engineers'}, {'id': 17, 'name': 'Data Engineer'}, {'id': 18, 'name': 'Product Manager'}, {'id': 19, 'name': 'UX Designer'}, {'id': 20, 'name': 'Solutions Architect'}, {'id': 21, 'name': 'Mobile Developer'}, {'id': 22, 'name': 'Machine Learning Engineer'}, {'id': 23, 'n

In [21]:
all_profile_ids = list(all_profiles.keys())

## Getting recommendations

The job recommendation service expects 

* a source profile ID (available in user_profiles above)
* a list of target profile IDs (available in all_profile_ids)
* a method (see below)
* a graph_tag associated with the graph containing the skills used in the profiles

There are two methods available; distance and duration. Distance is relatively simple and always available. Duration is an experimental alternative that requires scored courses for each skill. 

In [34]:
data = {"source_profile":user_profiles["Adam"],
        "target_profiles":all_profile_ids,
        "method":"distance",
        "graph_tag":graph_tag}

r = requests.post(base_url + "/job/recommendation", json=data)
print("Status code", r.status_code)

results = r.json()["profiles"]

Status code 200


In [35]:
# Fix type of profile ids (JSON converts keys to strings) and sort by distance

results = [(int(profile_id),result) for profile_id,result in results.items()]
results.sort(key=lambda x:x[1]["distance"])

In [48]:
print("{0:<12}|{1:<45}|{2:<3}".format("Profile ID", "Profile", "Distance"))
for profile_id,result in results:
    print("{0:<12}|{1:<45}|{2:<3}".format(profile_id, all_profiles[int(profile_id)], result["distance"]))

Profile ID  |Profile                                      |Distance
17          |Data Engineer                                |0  
20          |Solutions Architect                          |12 
10          |Business Intelligence Analyst                |13 
14          |Database Administrators                      |13 
3           |Computer and Information Systems Managers    |15 
5           |Software Engineer                            |16 
13          |Sales Engineer                               |16 
1           |Applications Engineer                        |17 
15          |Security Engineer                            |17 
18          |Product Manager                              |18 
21          |Mobile Developer                             |19 
7           |Systems Engineer                             |21 
24          |Data Analyst                                 |21 
28          |Database Architects                          |21 
26          |Systems Administrator                

During setup we used job profiles to create the users. The user above, Adam, was given the competences from the Data Engineer profile. As the competences in the user profile and Data Engineer profile are the same the distance is 0.